<a href="https://colab.research.google.com/github/ErkanGENC/Brain_detection/blob/main/T%C3%BCm%C3%B6r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
from google.colab import drive
from tensorflow.keras.utils import load_img
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,BatchNormalization,Dropout,GlobalAveragePooling2D

drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [96]:
cd /gdrive/MyDrive/Brain-Tumor-Classification-DataSet-master

[Errno 2] No such file or directory: '/gdrive/MyDrive/Brain-Tumor-Classification-DataSet-master'
/content


In [97]:
os.listdir()
os.getcwd()

'/content'

In [98]:
url = "/content/gdrive/.shortcut-targets-by-id/1qvAeI1c-3Xpfbkt1d8JeXMAPPyV2dZzj/Brain-Tumor-Classification-DataSet-master"

train_img = []
train_label = []

etiket = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']

trainUrl = os.path.join(url,"Training")
testUrl = os.path.join(url,"Testing")

for x in etiket:
  trainUrl2 = os.path.join(trainUrl,x)
  for y in os.listdir(trainUrl2):
    img = cv2.imread(os.path.join(trainUrl2,y))
    img = cv2.resize(img,(128,128))
    train_img.append(img)
    train_label.append(x)


for x in etiket:
  testUrl2 = os.path.join(testUrl,x)
  for y in os.listdir(testUrl2):
    img = cv2.imread(os.path.join(testUrl2,y))
    img = cv2.resize(img,(128,128))
    train_img.append(img)
    train_label.append(x)


train_img = np.array(train_img)
train_label = np.array(train_label)


In [99]:
from google.colab.patches import cv2_imshow

In [100]:
#for x in etiket:
#  testUrl2 = os.path.join(testUrl,x)
#  for y in os.listdir(testUrl2):
#    img = load_img(os.path.join(testUrl2,y))
#    img = cv2.imread(os.path.join(testUrl2,y))
#    cv2_imshow(img)

In [101]:
train_img.shape

(3264, 128, 128, 3)

In [102]:
train_label.shape

(3264,)

In [103]:
train_img,train_label = shuffle(train_img,train_label,random_state=12)

In [104]:
train_label.shape

(3264,)

In [105]:
train_img,test_img,train_label,test_label = train_test_split(train_img,train_label,test_size=0.3)

In [106]:
print(train_img.shape)
print(train_label.shape)
print(test_img.shape)
print(test_label.shape)

(2284, 128, 128, 3)
(2284,)
(980, 128, 128, 3)
(980,)


In [107]:
trainLabelNew = []
testLabelNew = []

for i in train_label:
    trainLabelNew.append(etiket.index(i))
train_label = trainLabelNew
train_label = to_categorical(train_label)


testLabelNew = []
for i in test_label:
    testLabelNew.append(etiket.index(i))
test_label = testLabelNew
test_label = to_categorical(test_label)


In [108]:
print(train_img.shape)
print(train_label.shape)
print(test_img.shape)

(2284, 128, 128, 3)
(2284, 4)
(980, 128, 128, 3)


In [109]:
test_label.shape

(980, 4)

## Modeller

### DenseNet

In [110]:
modelDenseNet = Sequential()


modelDenseNet.add(Conv2D(64,kernel_size=(3,3),activation="relu",input_shape=(128,128,3)))
modelDenseNet.add(BatchNormalization())
modelDenseNet.add(MaxPooling2D((2,2)))
modelDenseNet.add(Conv2D(128,kernel_size=(3,3),activation="relu"))
modelDenseNet.add(BatchNormalization())
modelDenseNet.add(MaxPooling2D(2,2))
modelDenseNet.add(Conv2D(256,kernel_size=(3,3),activation="relu"))
modelDenseNet.add(BatchNormalization())
modelDenseNet.add(GlobalAveragePooling2D())

modelDenseNet.add(Flatten())

modelDenseNet.add(Dense(256,activation="relu"))
modelDenseNet.add(Dropout(0.5))
modelDenseNet.add(Dense(4,activation="softmax"))

In [111]:
modelDenseNet.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")

In [112]:
modelDenseNet.fit(train_img,train_label,epochs=30,batch_size=20,validation_split=0.2)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.5423 - loss: 1.1335 - val_accuracy: 0.1926 - val_loss: 4.3079
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.6426 - loss: 0.9174 - val_accuracy: 0.5361 - val_loss: 1.2971
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.7267 - loss: 0.7152 - val_accuracy: 0.3742 - val_loss: 3.2357
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7584 - loss: 0.6554 - val_accuracy: 0.3304 - val_loss: 5.3474
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.7521 - loss: 0.6067 - val_accuracy: 0.4595 - val_loss: 2.7367
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7899 - loss: 0.5515 - val_accuracy: 0.4201 - val_loss: 1.6368
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.8165 - loss: 0.4826 - val_accuracy: 0.3545 - val_loss: 3.4915
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.8195 - loss: 0.4927 - val_accuracy: 0.4858 - 

In [113]:
loss1,accuracy1 = modelDenseNet.evaluate(test_img,test_label)

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7942 - loss: 0.6530


### ResNet

In [114]:
modelResnet = Sequential()


modelResnet.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128,128, 3)))
modelResnet.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
modelResnet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

modelResnet.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
modelResnet.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
modelResnet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

modelResnet.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
modelResnet.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
modelResnet.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
modelResnet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

modelResnet.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
modelResnet.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
modelResnet.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
modelResnet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

modelResnet.add(Flatten())

modelResnet.add(Dense(512, activation='relu'))
modelResnet.add(Dropout(0.5))

modelResnet.add(Dense(512, activation='relu'))
modelResnet.add(Dropout(0.5))

modelResnet.add(Dense(4, activation='softmax'))


modelResnet.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")

history = modelResnet.fit(train_img,train_label,epochs=10,validation_split=0.2,batch_size=128)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 12s 485ms/step - accuracy: 0.2546 - loss: 2.8258 - val_accuracy: 0.3063 - val_loss: 1.3553
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 204ms/step - accuracy: 0.3865 - loss: 1.3073 - val_accuracy: 0.4070 - val_loss: 1.4433
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 213ms/step - accuracy: 0.4501 - loss: 1.2424 - val_accuracy: 0.4311 - val_loss: 1.1890
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 215ms/step - accuracy: 0.4908 - loss: 1.1533 - val_accuracy: 0.5295 - val_loss: 1.0306
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 207ms/step - accuracy: 0.5356 - loss: 0.9923 - val_accuracy: 0.6061 - val_loss: 0.9392
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 209ms/step - accuracy: 0.6061 - loss: 0.8658 - val_accuracy: 0.6433 - val_loss: 0.8019
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 210ms/step - accuracy: 0.6543 - loss: 0.7876 - val_accuracy: 0.6543 - val_loss: 0.7818
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 211ms/step - accuracy: 0.6556 - loss: 0.7725 - val_accuracy: 0

In [115]:
loss2,accuracy2 = modelResnet.evaluate(test_img,test_label)

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.7257 - loss: 0.7961


### AlexNet

In [116]:
modelAlexnet = Sequential()


modelAlexnet.add(Conv2D(96,kernel_size=(11,11),strides=(4,4),activation="relu",input_shape=(128,128,3)))

modelAlexnet.add(MaxPooling2D(pool_size=(3, 3), strides= (2, 2)))
modelAlexnet.add(Conv2D(256,kernel_size=(5,5),strides=(1,1),activation="relu",padding="same"))

modelAlexnet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
modelAlexnet.add(Conv2D(384,kernel_size=(3,3),strides=(1,1),activation="relu",padding="same"))

modelAlexnet.add(Conv2D(384,kernel_size=(3,3),strides=(1,1),activation="relu",padding="same"))

modelAlexnet.add(Conv2D(256,kernel_size=(3,3),strides=(1,1),activation="relu",padding="same"))

modelAlexnet.add(MaxPooling2D(pool_size=(1, 1), strides=(2, 2)))

modelAlexnet.add(Flatten())

modelAlexnet.add(Dense(4096,activation="relu"))
modelAlexnet.add(Dropout(0.5))
modelAlexnet.add(Dense(4096,activation="relu"))
modelAlexnet.add(Dropout(0.5))
modelAlexnet.add(Dense(4,activation="softmax"))

In [117]:
modelAlexnet.compile(metrics=["accuracy"],loss="categorical_crossentropy",optimizer="adam")

In [118]:
history = modelAlexnet.fit(train_img,train_label,epochs=10,batch_size=128,validation_split=0.2)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 307ms/step - accuracy: 0.3068 - loss: 41.7167 - val_accuracy: 0.2691 - val_loss: 1.6202
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.4092 - loss: 1.3001 - val_accuracy: 0.4530 - val_loss: 1.2631
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.5091 - loss: 1.1318 - val_accuracy: 0.4792 - val_loss: 1.0384
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5389 - loss: 1.0314 - val_accuracy: 0.4836 - val_loss: 1.1721
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.5334 - loss: 1.0552 - val_accuracy: 0.5930 - val_loss: 0.9657
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.6381 - loss: 0.8678 - val_accuracy: 0.4223 - val_loss: 1.2854
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5748 - loss: 1.0052 - val_accuracy: 0.5821 - val_loss: 0.9110
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.6352 - loss: 0.8229 - val_accuracy: 0.6499 -

In [119]:
loss2,accuracy2 = modelAlexnet.evaluate(test_img,test_label)

31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.6536 - loss: 0.7702


### LeNet

In [120]:
modelLenet = Sequential()

modelLenet.add(Conv2D(filters=6,kernel_size=(5,5),padding="valid",activation="relu",input_shape=(128,128,3)))
modelLenet.add(MaxPooling2D(pool_size=(2,2)))
modelLenet.add(Conv2D(filters=16,kernel_size=(5,5),activation="relu"))
modelLenet.add(MaxPooling2D(pool_size=(2,2)))

modelLenet.add(Flatten())

modelLenet.add(Dense(120,activation="relu"))

modelLenet.add(Dense(4,activation="softmax"))

modelLenet.compile(metrics=["accuracy"],optimizer="adam",loss="categorical_crossentropy")

In [121]:
history = modelLenet.fit(train_img,train_label,epochs=10,batch_size=128,validation_split=0.2)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 199ms/step - accuracy: 0.3043 - loss: 193.3338 - val_accuracy: 0.3326 - val_loss: 6.0187
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3652 - loss: 3.7617 - val_accuracy: 0.4814 - val_loss: 1.3081
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.4824 - loss: 1.2030 - val_accuracy: 0.5317 - val_loss: 1.1428
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5784 - loss: 1.0320 - val_accuracy: 0.5624 - val_loss: 1.0602
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6273 - loss: 0.9095 - val_accuracy: 0.5733 - val_loss: 0.9698
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6762 - loss: 0.7786 - val_accuracy: 0.6324 - val_loss: 0.9112
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7547 - loss: 0.6264 - val_accuracy: 0.6761 - val_loss: 0.8307
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8342 - loss: 0.4794 - val_accuracy: 0.6871 

In [122]:
loss,accuracy = modelLenet.evaluate(test_img,test_label)

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7553 - loss: 0.7079
